<a href="https://colab.research.google.com/github/edgeofchaos42/ComplexityExplorer/blob/main/Session_9_Traders_move_part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1- Review lesson 8 and intro

Script Notes: 

In lesson 8 we introduced the sugar and spice step functions and in this lesson we begin the trader functions which will take up the next several lessons. 

Remember if you shut down your colab kernel you will need to add the sugar-map.txt file again or you can mount your drive and not have to load it every time   

*Review each class and model set up*

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
try: 
  import mesa
except: 
  !pip install mesa --quiet
  import mesa
import numpy as np
import matplotlib.pyplot as plt



%matplotlib inline 

     |████████████████████████████████| 1.8 MB 7.5 MB/s 
     |████████████████████████████████| 66 kB 4.6 MB/s 


In [3]:
class Sugar(mesa.Agent):
    """
    Sugar is a FSM that
    - contains an amount of sugar
    - grows 1 amount of sugar at each turn (rule G1).
    """
    
    
    def __init__(self, unique_id, model, pos, max_sugar): 
      super().__init__(unique_id, model) #part 3
      self.pos = pos
      self.amount = max_sugar
      self.max_sugar = max_sugar

    def step(self): #Part 1
      self.amount = min([self.max_sugar,self.amount+1])
      
      

In [4]:
class Spice(mesa.Agent):
    """
    Spice is a FSM that
    - contains an amount of spice
    - grows 1 amount of spice at each turn. (rule G1)
    """

    def __init__(self, unique_id, model, pos, max_spice): 
        super().__init__(unique_id, model) 
        self.pos = pos
        self.amount = max_spice
        self.max_spice = max_spice

    def step(self): #Part 1
      self.amount = min([self.max_spice,self.amount+1])
       

## Add agent move move function 

**Part 1**

------------

So we added the call from the main model so lets build a simple move function just to make sure it calls and is moving agents. 

No we know our move function is being called we can start the rather complicated process of detemrining as an agent wehre to move

To move we want to 
--see all our options, based on our vision using left, right, up down of our von neuman neighborhood. 
--calculate which one is the best
--then move there

For this session we will just do see all the options. 

**Part 2**

------------

To do this we will make a list of all the cells in our vision and use Mesa's get_neighborhood function and python's list concantenation. Which you can see at [line 243](https://github.com/projectmesa/mesa/blob/de88728f6c2e28bb7e88170b47618c627b586c0d/mesa/space.py#L243) in the space.py code. This function has 5 parameters with various defaults. First you need to pass in a position which is just the agents position, next is a boolean for whether or not we are using the moore neighborhood as this is an attirbute of the agent we can just pass in self.moore. Next is another boolean whether or not to include the position the agent is in. Per growing artifical societies we say True,finally an integer for the radius the agent can see which is the agent vision. 

now lets do a print statement to make sure it works. As you can see Mesa returns a list of tuples which are the grid positions that meet the criteria radius and type of neighborhood. 

**Part 3**

------------

However there is a complication. Per Growing Artifcial Societies, they cannot move to a cell if a Trader agent is already there. This is one more example where if we kept the agent order the same everytime it would change the results. So we will create a special function to check if a Trader is in the cell. called is_occupied

We will then pass in the position and use mesa get_cell_list_content function which is [line 387](https://github.com/projectmesa/mesa/blob/de88728f6c2e28bb7e88170b47618c627b586c0d/mesa/space.py#L387) in space.py
and as you can see it accepts an iterable such as a tuple and returns the contents in a list. We must also exclude the position the agent is at as we can stay at this position.  

So we will then iterate through that list and if the object is a Trader we will return True and if not we will return false. 

if a trader is in the list so True we will not add it. 


In [11]:
class Trader(mesa.Agent):
    """
    TraderAgent is a 
    - has a metabolism for sugar and spice
    - harvest and trades sugar and spice to survive and thrive
    """

    def __init__(self, unique_id, model,pos,moore=False, sugar=0, 
                 spice=0, metabolism_sugar=0, metabolism_spice=0,
                 vision=0): 
      super().__init__(unique_id, model)
      self.pos = pos 
      self.moore = False 
      self.sugar = sugar 
      self.spice = spice 
      self.metabolism_sugar=metabolism_sugar 
      self.metabolism_spice=metabolism_spice 
      self.vision = vision 

    #part 3
    def is_occupied_by_other(self, pos): 
      if pos == self.pos: 
        # We consider the agent's own position to be "unoccupied" in
        # the sense that it is one of the legal positions the agent can occupy.
        return False
      this_cell = self.model.grid.get_cell_list_contents(pos)
      for a in this_cell:
          # Occupied by other agent
          if isinstance(a, Trader) and a.pos != self.pos:
              return True
      return False
       
    def move(self): 

      #print("I am agent {} and am about to move".format(self.unique_id)) #part 1
      
      # 1. Get neighbors within vision

      neighbors = [i 
                   for i in self.model.grid.get_neighborhood(
                       self.pos, self.moore,True, self.vision
                       )
                   if not self.is_occupied_by_other(i)
      ]

      print(len(neighbors), self.vision)



## Add Trader randomize and move

**Part 1:**

-----------

To begin we will ad the move traders move to to main model class sugascapeg1mt.

As we do not want to give any agent a mover advantage, for example if the same agent goes first all the time they will always get the prime sugar and spice point in their vision. We need to randomize the agents. 

As Mesa is mostly in python you can see the code and underlying data structures easily. RandomActivationByType stores all agents in a dictionary by type we can see this again with some simple print functions and running the model for one step. 

Now a challenge with traders is agents can die and this creates a lot of complications as the datastrcutures are being updated as agents move. For example, you can not change a dictionary, but removing agent 4038, as that will result in a runtime error. So will create the trader agents put them in a list and then we will shuffle the list multiple time to prevent mover advantage as dmeonstrated in Ken Comer's seminal dissertation -- [Who Goes First](http://mars.gmu.edu/bitstream/handle/1920/9070/Comer_gmu_0883E_10539.pdf).

This may seem a little redundant at first but as we do all the parts of traders you will see why it is important. 

so first we get just the traders then we put them in a list then we shuffle the list using existing python randomization functions. We then iterate through this list and the move function, which we need to build. 





In [12]:
class SugarscapeG1mt(mesa.Model):

  def __init__(self, width=50, height=50, initial_population=100,
               endowment_min =25, endowment_max =50, metabolism_min = 1,
                 metabolism_max = 5, vision_min = 1, vision_max =5): 
    
    self.width = width
    self.height = height
    self.initial_population =initial_population 
    self.endowment_min = endowment_min 
    self.endowment_max = endowment_max 
    self.metabolism_min = metabolism_min
    self.metabolism_max = metabolism_max
    self.vision_min = vision_min
    self.vision_max = vision_max


    self.schedule = mesa.time.RandomActivationByType(self)
    self.grid = mesa.space.MultiGrid(self.width, self.height, torus=False)

    sugar_distribution = np.genfromtxt("/content/drive/MyDrive/sugar-map.txt") 
    spice_distribution = spice = np.flip(sugar_distribution,1)
    
    #ensure unique id
    agent_id = 0
    for _, x, y in self.grid.coord_iter(): 
      max_sugar = sugar_distribution[x,y]
      if max_sugar > 0: 
        sugar = Sugar(agent_id, self, (x,y),max_sugar)
        self.grid.place_agent(sugar, (x, y))
        #Place Agent
        self.schedule.add(sugar) 
        agent_id += 1

      max_spice = spice_distribution[x,y]
      if max_spice > 0: 
        spice = Spice(agent_id, self, (x,y), max_spice)
        self.grid.place_agent(spice,(x,y))
        self.schedule.add(spice)
        agent_id += 1
         
    
    for i in range(self.initial_population):
      #get agent position
      x = self.random.randrange(self.width) 
      y = self.random.randrange(self.height) 
      # See GAS page 108 for parameters initialization.
      # Each agent is endowed by a random amount of sugar and spice
      sugar = self.random.uniform(self.endowment_min, self.endowment_max+1) 
      spice = self.random.uniform(self.endowment_min, self.endowment_max+1) 
      #add metabolism also on page #108
      metabolism_sugar = self.random.uniform(self.metabolism_min, self.metabolism_max)
      metabolism_spice = self.random.uniform(self.metabolism_min, self.metabolism_max)
      #add vision
      vision =int(self.random.uniform(self.vision_min, self.vision_max))
      trader = Trader(
                agent_id,
                self,
                (x, y),
                False,
                sugar,
                spice,
                metabolism_sugar,
                metabolism_spice,
                vision,
            )
      self.grid.place_agent(trader, (x, y))
      self.schedule.add(trader)
      agent_id += 1

  def step(self): 
    #part 2
    #print(self.schedule.agents_by_type.keys())
    #print(self.schedule.agents_by_type[Trader])
    
    for sugar in self.schedule.agents_by_type[Sugar].values(): 
      sugar.step()
    
    for spice in self.schedule.agents_by_type[Spice].values(): 
      spice.step()
    #part 1
    Traders = self.schedule.agents_by_type[Trader].values()
    Trader_shuffle = list(Traders)
    self.random.shuffle(Trader_shuffle)
    for agent in Trader_shuffle: 
      agent.move()  
    
    self.schedule.time +=1
    self.schedule.steps += 1
    

  
  def run_model(self, step_count=1000):
    
    for i in range(step_count):
      self.step()
      

    #self.schedule.step()_

        

## Model Run 


In [13]:
model = SugarscapeG1mt()
model.run_model(step_count=1) #part 1



11 2
13 2
22 3
5 1
13 2
32 4
24 3
12 2
5 1
13 2
34 4
5 1
5 1
4 1
25 3
23 3
5 1
13 2
5 1
25 3
41 4
23 3
24 3
24 3
25 3
40 4
12 2
5 1
3 1
5 1
13 2
5 1
12 2
41 4
11 2
5 1
4 1
25 3
13 2
13 2
13 2
13 2
37 4
31 4
39 4
5 1
25 3
39 4
5 1
5 1
5 1
12 2
5 1
24 3
24 3
38 4
40 4
25 3
40 4
40 4
40 4
25 3
38 4
13 2
5 1
12 2
38 4
5 1
38 4
37 4
38 4
39 4
24 4
39 4
40 4
24 3
41 4
25 3
23 3
4 1
5 1
12 2
5 1
13 2
13 2
39 4
40 4
4 1
12 2
24 3
5 1
38 4
23 3
23 3
25 3
31 4
23 3
24 3
5 1
25 3
